In [1]:
!pip install langchain langchain-community sentence-transformers faiss-cpu

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import pandas as pd
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 74.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt

In [2]:
available_categories = ['Уход за волосами', 'Уход за лицом', 'Уход за телом', 'Макияж', 'Маникюр и педикюр', 'Парфюмерия', 'Загар и защита от солнца', 'Мужская косметика', 'Детская косметика и парфюмерия', 'Беременным и кормящим', 'Аппаратная косметология и массаж', 'Ароматерапия', 'Оборудование и материалы для тату-салона', 'Мебель и оборудование для салонов красоты', 'Для кошек', 'Для собак', 'Для грызунов и хорьков', 'Для птиц', 'Для рыб и рептилий', 'Для лошадей', 'Ветаптека', 'Фермерское хозяйство', 'Ветеринарное оборудование', 'Лекарственные средства', 'Витамины и БАДы', 'Парафармацевтика', 'Оптика', 'Медицинские приборы', 'Медицинское оборудование', 'Аксессуары и комплектующие', 'Медицинские изделия и расходные материалы', 'Одежда и текстиль медицинские', 'Медицинские инструменты', 'Материалы и средства стоматологические', 'Товары для гигиены', 'Товары для реабилитации', 'Ортопедия', 'Ручные массажеры и аппликаторы', 'Мебель медицинская', 'Женщинам', 'Мужчинам', 'Детям', 'Уход и аксессуары', 'Бытовая химия', 'Личная гигиена', 'Туризм и отдых на природе', 'Рыбалка', 'Охота и стрельба', 'Лодки и лодочные моторы', 'Одежда для рыбалки и охоты', 'Обувь для рыбалки и охоты', 'Тактическая одежда и аксессуары', 'Наборы для выживания', 'Автомобили новые', 'Климатическая техника', 'Техника для кухни', 'Техника для дома', 'Техника для красоты и здоровья', 'Крупная бытовая техника', 'Техника для общепита', 'Женщинам', 'Мужчинам', 'Детям', 'Униформа и рабочая одежда', 'Уход за одеждой', 'Электронные сертификаты', 'Онлайн-подписки', 'Игры и игровые подписки', 'Программное обеспечение', 'Облачные хранилища', 'Услуги', 'Садовая мебель', 'Компьютерная и офисная мебель', 'Мягкая мебель', 'Столы и стулья', 'Мебель для хранения', 'Мебель для спальни и комплектующие', 'Мебель для кухни', 'Мебель для ванной', 'Бескаркасная мебель', 'Детская мебель', 'Мебель для бизнеса', 'Сейфы и архивные шкафы', 'Телефоны и смарт-часы', 'Наушники и аудиотехника', 'Ноутбуки, планшеты и электронные книги', 'Телевизоры и видеотехника', 'Комплектующие для ПК', 'Компьютеры и периферия', 'Игровые приставки и ноутбуки', 'Офисная техника', 'Фото и видеокамеры', 'Охранные системы и видеонаблюдение', 'Умный дом', 'Квадрокоптеры и аксессуары', 'Навигаторы', 'Оптические приборы', 'Часы и электронные будильники', 'Аксессуары для электроники', 'Запчасти для легковых автомобилей', 'Автоаксессуары и принадлежности', 'Масла и автохимия', 'Шины и диски', 'Автозвук', 'Транспортные средства', 'Уход за автомобилем', 'Электроника для автомобиля', 'Инструменты и оборудование', 'Гараж и автосервис', 'Аккумуляторы и аксессуары', 'Мототовары', 'Запчасти для грузовиков и спецтехники', 'Автолитература', 'Коллекционирование', 'Посуда и утварь', 'Предметы военной истории', 'Предметы интерьера', 'Предметы быта', 'Мебель', 'Украшения', 'Часы наручные', 'Канцелярия', 'Приборы и инструменты', 'Живопись и графика', 'Виниловые пластинки', 'Галантерея', 'Товары для курения', 'Музыкальные инструменты', 'Религиозная атрибутика', 'Кольца', 'Серьги', 'Браслеты', 'Украшения на шею', 'Комплекты украшений', 'Шармы', 'Броши и значки', 'Пирсинг', 'Часы', 'Детские украшения', 'Запонки и зажимы', 'Сувениры', 'Шнурки', 'Аксессуары для волос', 'Религиозные украшения', 'Нехудожественная литература', 'Бизнес-литература', 'Художественная литература', 'Детям и родителям', 'Учебная литература', 'Электронные книги', 'Букинистика', 'Комиксы', 'Манга', 'Печать по требованию', 'Подарочные издания книг', 'Журналы и газеты', 'Литература на иностранных языках', 'Электронные аудиокниги', 'Подписки на электронные книги и аудиокниги', 'Антикварные книги', 'Книги б/у', 'Блокноты и ежедневники', 'Буклеты, листовки и брошюры', 'Товары для школы и обучения', 'Игрушки и игры', 'Спорт и игры на улице', 'Подгузники и гигиена', 'Детская комната', 'Коляски и автокресла', 'Детское питание', 'Товары для кормления', 'Товары для мам', 'Посуда и кухонные принадлежности', 'Текстиль', 'Освещение', 'Декор и интерьер', 'Дача и сад', 'Цветы, растения и горшки', 'Хозяйственные товары', 'Хранение вещей', 'Товары для бань и саун', 'Товары для праздников','Религия и эзотерика', 'Ритуальные товары', 'Женщинам', 'Мужчинам', 'Детям', 'Путешествия', 'Игровые жанры', 'PlayStation', 'Nintendo', 'Xbox', 'Игровые приставки', 'PC', 'Mobile gaming', 'Ретро-консоли', 'Игровые картриджи', 'Игровая атрибутика', 'Игровые наушники', 'Геймпады', 'Очки виртуальной реальности', 'Рули и педали', 'Аксессуары для игровых приставок', 'Аксессуары для киберспорта', 'Аккумуляторы и зарядки', 'Сумки и чехлы', 'Запчасти для игровых консолей', 'Музыка', 'Виниловые пластинки', 'Видео', 'Письменные принадлежности', 'Бумага', 'Бумажная продукция', 'Папки и файлы', 'Обложки', 'Демонстрационные доски', 'Офисные принадлежности', 'Подставки и визитницы', 'Печати и штампы', 'Калькуляторы', 'Картриджи для лазерных принтеров', 'Картриджи для струйных принтеров', 'Чертежные принадлежности', 'Оборудование и принадлежности для торговли', 'Полиграфическое оборудование', 'Сублимационная печать']

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model_kwargs = {'device': device}
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs)

/tmp/ipykernel_23/71589224.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [ ]:
available_category_vectors = embedding_model.embed_documents(available_categories)

In [ ]:
faiss_index = FAISS.from_texts(available_categories, embedding_model)

In [ ]:
import json

# matching
predictions_path = "/kaggle/input/llm-rec-final/pairs_final.json"
new_predictions_path = "predicted_categories.json"
predicted_categories_map = {}

with open(predictions_path, "r", encoding="utf-8") as f:
    predictions = json.load(f)

for i, prediction in tqdm(enumerate(predictions), total=len(predictions), desc="Processing Predictions"):
    categories_to_match = prediction[2].split("; ")
    prediction[-1] = categories_to_match
    arr = []
    for cat in categories_to_match:
        arr.append(faiss_index.similarity_search(cat, k=1)[0].page_content)
    prediction.append(arr)
    predictions[i] = prediction

with open(new_predictions_path, "w", encoding="utf-8") as file:
    json.dump(predictions, file, ensure_ascii=False, indent=4)

print(predictions)

In [ ]:
# remove cycles
with open("predicted_categories.json", "r", encoding="utf-8") as f:
    predictions = json.load(f)

count = 0
for i,pred in enumerate(predictions):
    filtered_arr = [c for c in pred[3] if c != pred[0] and c != pred[1]]
    pred[3] = filtered_arr
    predictions[i] = pred
    if (pred[0] in pred[2] or pred[1] in pred[2] or pred[0] in pred[3] or pred[1] in pred[3]):
        count += 1
        print(filtered_arr)

with open("predicted_categories.json", "w", encoding="utf-8") as file:
    json.dump(predictions, file, ensure_ascii=False, indent=4)